<a href="https://colab.research.google.com/github/suplab/amazon-bedrock-genai-labs/blob/main/02_Amazon_Bedrock_Knowledge_Bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Bedrock Knowledge Bases

Amazon Bedrock Knowledge Bases enables organizations to enhance foundation models with private data without model fine-tuning. This unit explores how Knowledge Bases implements Retrieval-Augmented Generation (RAG) to provide contextually relevant responses based on your organization's proprietary information.

### Why Knowledge Bases Matter
Imagine you're part of a DevOps team with dozens of internal runbooks documenting how to handle everything from restarting staging environments to debugging deployment pipelines. What if you could turn all that internal knowledge into something queryable? A place where engineers could ask natural questions like "How do I restart staging?" and get reliable answers backed by your actual documentation.

That's exactly what Amazon Bedrock Knowledge Bases enables - a way to make your private content accessible to foundation models without having to fine-tune them.

### How Knowledge Bases Works: The RAG Approach
Knowledge Bases uses a technique called Retrieval-Augmented Generation (RAG). Here's how it works:

1. Document Processing

    * Upload your documents to Amazon S3
    * The system creates embeddings (numerical representations of text) using models like Amazon Titan Text Embeddings
    * Documents are chunked into manageable pieces

2. Vector Storage

    * Those embeddings get stored in a vector database (like Amazon OpenSearch Serverless)
    * This enables semantic searching - finding content based on meaning, not just keywords

3. Query Processing

    * When someone asks a question:
      - Their question gets converted to an embedding
      - The system finds similar vectors in the database based on semantic search
      - A foundation model generates a response using the retrieved content
      - Responses include citations back to source documents

### Setting Up Your Knowledge Base

**Initial Setup**  
When creating a new knowledge base, you'll need to provide a name and Bedrock will handle the creation of necessary IAM roles to enable interactions between services like Amazon S3 and Amazon OpenSearch Service.

**Configure Data Source**  
Your knowledge base needs data to work with. While the demo uses Amazon S3 as the primary data source for storing runbooks, Bedrock supports multiple data sources including Confluence, Salesforce, SharePoint, and even custom sources. You'll need to specify the location of your documents and give your data source a name.

**Choose Parsing Strategy**  
The parsing strategy determines how Bedrock extracts content from your documents. The default parser handles common file types like text and PDFs, while advanced options like Bedrock Data Automation can handle multimodal content including charts and images. Choose based on your document types and extraction needs.

**Select Chunking Strategy**.
Chunking is crucial for effective retrieval. The default strategy breaks content into 300-token chunks while preserving sentence boundaries. Whether you choose default, fixed size, hierarchical, semantic, or no chunking depends on your document structure and how you want information to be retrieved.

**Configure Models and Storage**  
Your knowledge base needs two key components: an embedding model to convert text into vectors (like Amazon Titan Text Embeddings v2) and a vector database to store these embeddings (like Amazon OpenSearch Serverless). These work together to enable semantic search capabilities.

**Final Steps**  
After reviewing and creating your knowledge base, you'll need to sync your data source to generate embeddings. Once syncing is complete, you can start using the RetrieveAndGenerate API to query your knowledge base and get responses based on your private content.

In [ ]:
!pip install boto3

In [ ]:
import boto3

bedrock = boto3.client(service_name='bedrock-agent-runtime', region_name='us-east-1')

response = bedrock.retrieve_and_generate(
     input={"text": "How do I restart the staging environment?"},
     retrieveAndGenerateConfiguration={
          "type": "KNOWLEDGE_BASE",
          "knowledgeBaseConfiguration": {
               "knowledgeBaseId": "<KNOWLEDGE_BASE_ID>",
                "modelArn": f"arn:aws:bedrock:us-east-1:foundation-model/<MODEL_ID>"
             }
        }
    )

print(response)


In [ ]:
output_text = response.get("output", {}).get("text", "")
citations = []

for citation in response.get("citations", []):
    span = citation.get("generatedResponsePart", {}).get("textResponsePart", {}).get("span", {})
    reference = citation.get("retrievedReferences", [{}])[0]
    uri = reference.get("location", {}).get("sJLocation", {}).get("uri", "")
    chunk_text = reference.get("content", {}).get("text", "")

    citations.append({
        "cource_uri": uri,
        "start": span.get("start"),
        "end": span.get("end"),
        "excerpt": chunk_text[:200] + "..." if chunk_text else "N/A"
    })

print(output_text)
print(citations)

## Prompt Management
Amazon Bedrock Prompt Management is a feature that enables developers to create, version, and reuse prompts across different workflows and applications. Instead of embedding prompts directly in application code, Prompt Management provides a centralized way to manage and optimize prompts for foundation models.



### What is Prompt Management?
Prompt Management allows you to:

* Create and save prompts as reusable templates
* Version control your prompts
* Include variables (placeholders) in prompts
* Manage prompts across different environments (development, staging, production)

### Key components



#### Prompt Structure
A prompt in Bedrock Prompt Management requires:

* Name
* One or more variants
* Template type (chat or text)
* Model configurations

#### Variants
Each variant includes:

* Variant name
* Model ID
* Inference configuration (temperature, topP, stopSequences)
* Template configuration

#### Template Types
Two main template types:

* **Chat Template:** For conversational formats, includes system prompts and tool configurations
* **Text Template:** For single text message prompts

You set the template type for the prompt by configuring the templateType parameter when you create the prompt. Valid values are TEXT or CHAT. CHAT is only compatible with models that support the
Converse API. If you want to use prompt caching, you must use the CHAT template type.



### Initialize Bedrock and Set Defaults

In [7]:
import boto3
import json
import datetime
from IPython.display import display, JSON

bedrock_agent = boto3.client("bedrock-agent", region_name="us-east-1")
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name="us-east-1")

model_id = "amazon.nova-micro-v1:0"

### Create a Prompt Template

In [ ]:
prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        # First variant
        {
            "name": "summary",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7, # Higher temperature for more focused responses
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        },
        # Second variant
        {
            "name": "detailed",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.2, # Lower temperature for more focused responses
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])

**Expected output:** A JSON response object containing information about the created prompt. A **prompt ARN**.

```
==================== Response Object ====================
…
arn"arn:aws:bedrock:us-east-1:XXXXXXXXXXXX:prompt/0185LCGNHJ"
createdAt"2025-05-27T16:37:10.443054Z"
defaultVariant"v1"
description"Generates inclusive job descriptions from structured inputs"
id"0185LCGNHJ"
name"job-description-20250527123710"
updatedAt"2025-05-27T16:37:10.443054Z"
version"DRAFT"

==================== Prompt ARN ====================

arn:aws:bedrock:us-east-1:XXXXXXXXXXXX:prompt/0185LCGNHJ
```

### Create a New Version of the Prompt

In [ ]:
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier="<ARN-FROM-PREVIOUS_RESPONSE>"
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])

**Expected output:** A prompt ARN with a :X at the end of it which defines the version number.

```
==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:XXXXXXXXX:prompt/5K93WKPTGK:1
```

### Invoke a Model Using the Versioned Prompt Template

In [ ]:
prompt_arn_with_version = "ARN-FROM-PREVIOUS_RESPONSE:Version"

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])

**Expected output:** Output from the model defining the job description based off of the input.

```
==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**

We are seeking a talented and inclusive UX Designer to join our dynamic team in New York or remotely. As a UX Designer, you will play a pivotal role in designing user interfaces, conducting usability testing, and collaborating closely with our product teams to create exceptional user experiences.

**Responsibilities:**

- Design intuitive and engaging user interfaces that meet user needs and business goals.
- Conduct usability testing to gather insights and refine designs.
- Collaborate with cross-functional teams, including product managers, developers, and marketers, to ensure cohesive and user-centered design solutions.

**Requirements:**

- 3+ years of experience in UX design.
- Proficiency in Figma, HTML, and CSS.
- Strong communication skills to effectively convey ideas and collaborate with team members.
- Ability to work in a fast-paced, inclusive environment.

**Location:** New York or remote

**Work Type:** Full-time

Join us and contribute to a diverse and inclusive team dedicated to creating innovative products that make a difference. We celebrate diversity and are committed to creating an inclusive environment for all employees. If you are passionate about UX design and meet the requirements, we would love to hear from you!
```

### Optimize the Prompt

In [ ]:
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)

**Expected output **should be similar to the following:

```
==================== ORIGINAL PROMPT ====================
   
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words

==================== OPTIMIZED PROMPT ====================

"## Task
Write a professional and inclusive job description for the given role.

## Instructions
1. Review the provided inputs:
### Job Title
{{job_title}}

### Responsibilities
{{responsibilities}}

### Requirements
{{requirements}}

### Location
{{location}}

### Work Type
{{work_type}}

2. Follow these guidelines to create the job description:
- Start with a clear and concise summary of the role.
- Use inclusive language that avoids bias or discrimination.
- Keep the job description under 250 words.
- Organize the content into the following sections:

### Job Summary
[Provide a brief overview of the role, its purpose, and key responsibilities]

### Responsibilities
- [List the main responsibilities using bullet points]

### Requirements
- [List the essential requirements using bullet points]

### Additional Information
- Location: {{location}}
- Work Type: {{work_type}}

3. Ensure the job description is professional, well-structured, and easy to understand.
4. Avoid any preamble or additional information beyond the requested job description.

Now provide your response immediately without any preamble."
```

### Update Prompt and Create New Prompt Version

In [ ]:
prompt_identifier = "Original-Prompt-ARN from Section:Create a Prompt Template"

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = "<ARN From Previous Step>"


response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])

**Expected output:** A prompt ARN with a :X at the end, indicating the version number.

```
==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:XXXXXXXXX:prompt/5K93WKPTGK:2
```

### Compare Versions

In [ ]:
prompt_arn = "<First VersionedPrompt ARN>"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

In [ ]:
prompt_arn = "<Seconf VersionedPrompt ARN>"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

### Clean Up

In [ ]:
response = bedrock_agent.list_prompts()

for prompt in response['promptSummaries']:
    prompt_id = prompt['id']  # The correct key is 'id', not 'promptId'
    print(f"Deleting prompt: {prompt['name']} (ID: {prompt_id})")
    bedrock_agent.delete_prompt(promptIdentifier=prompt_id)

print("All prompts deleted successfully")

**Expected output:**

```
Deleting prompt: job-description-1231342342341231 (ID: XXXXXXXXXX)
Deleting prompt: job-description-1231232131243132 (ID: XXXXXXXXXX)
All prompts deleted successfully
```